In [ ]:
# %%

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

from utils import create_mixture, get_stft, pad_audio_center
from config import DATASET_NOISE_PATH, DATASET_PERCUSSION_PATH, DATASET_MIX_AUDIO_PATH

# Load noise metadata
metadata_noise = os.path.join(DATASET_NOISE_PATH, "UrbanSound8k.csv")
metadata_noise = pd.read_csv(metadata_noise)

# Change classID 8 to 6
metadata_noise['classID'] = metadata_noise['classID'].replace(8, 6)
# Sort by class ID and fold and reset index
metadata_noise = metadata_noise.sort_values(
    by=['classID', 'fold']).reset_index(drop=True)

# Get unique noise classes and their IDs
noise_class = metadata_noise['class'].unique()
noise_classID = metadata_noise['classID'].unique()

# Load percussion metadata
metadata_perc = os.path.join(DATASET_PERCUSSION_PATH, "metadata.csv")
metadata_perc = pd.read_csv(metadata_perc)

# Create a DataFrame for noise classes
noise_data = {
    'class': noise_class,
    'classID': noise_classID,
    'count': [len(metadata_noise[metadata_noise['class'] == c]) for c in noise_class],
}
df = pd.DataFrame(noise_data)

# Total number of percussion files
percussion_count = len(metadata_perc)

# Calculate total number of noise files
total_noise_files = df['count'].sum()

# Calculate proportions for each noise class
df['proportion'] = df['count'] / total_noise_files

# Number of noise files to select for each percussion file
noise_files_per_percussion = 5

# Create directories if they do not exist
os.makedirs(DATASET_MIX_AUDIO_PATH, exist_ok=True)

# we will save the informations of all the noised used for each percussion file
# save the informations in a csv file : percussion_file, noise_file, noise_class, mixture_file

In [ ]:
# %%

# Shuffle the noise files to ensure randomness
metadata_noise = metadata_noise.sample(frac=1, random_state=42).reset_index(drop=True)

# Create a list to keep track of the usage count for each noise file
noise_usage = {row['slice_file_name']: 0 for _, row in metadata_noise.iterrows()}

# Create directories if they do not exist
os.makedirs(DATASET_MIX_AUDIO_PATH, exist_ok=True)

# Store metadata information
metadata_info = []

# Round-robin approach to ensure all noise files are used
noise_files = metadata_noise['slice_file_name'].tolist()
noise_files_iter = iter(noise_files)

def get_next_noise_file():
    global noise_files_iter  # Move global declaration here
    try:
        return next(noise_files_iter)
    except StopIteration:
        # Restart the iterator if we've gone through all files
        noise_files_iter = iter(noise_files)
        return next(noise_files_iter)

# Iterate over each percussion file
for index, row in metadata_perc.iterrows():
    percussion_filename = row['name']
    percussion_file = os.path.join(DATASET_PERCUSSION_PATH, percussion_filename)

    # Create a list to store noise information
    sound_info = []

    for _ in range(50):  # Adjust this number as needed
        noise_file_name = get_next_noise_file()
        noise_file_row = metadata_noise[metadata_noise['slice_file_name'] == noise_file_name].iloc[0]
        selected_fold = noise_file_row['fold']
        noise_file = os.path.join(DATASET_NOISE_PATH, f"fold{selected_fold}", noise_file_name)
        noise_class = noise_file_row['class']

        # Update noise usage count
        noise_usage[noise_file_name] += 1

        # Save sound information
        sound_info.append({
            'percussion_file': percussion_filename,
            'noise_file': noise_file_name,
            'mix_file': f"{percussion_filename}_{noise_file_name}",
            'noise_class': noise_class,
            'fold': selected_fold,
        })

    # Save sound information to a DataFrame and append to metadata_info
    sound_info_df = pd.DataFrame(sound_info)
    metadata_info.append(sound_info_df)

    # Save sound information to a CSV file
    sound_info_df.to_csv(os.path.join(DATASET_MIX_AUDIO_PATH, f"{percussion_filename}_info.csv"), index=False)

# Concatenate all metadata info into a single DataFrame and save to a CSV file
metadata_info = pd.concat(metadata_info)
metadata_info.to_csv(os.path.join(DATASET_MIX_AUDIO_PATH, "metadata.csv"), index=False)

# Group by noise class to check the distribution of unique noise files per class
unique_noise_files_per_class = metadata_info.groupby('noise_class')['noise_file'].nunique()
print(unique_noise_files_per_class)

# Check the value counts of noise classes
noise_class_counts = metadata_info['noise_class'].value_counts()
print(noise_class_counts)

noise_class
air_conditioner     1000
car_horn             429
children_playing    1000
dog_bark            1000
drilling            1000
engine_idling       1000
jackhammer          1000
siren                929
Name: noise_file, dtype: int64
noise_class
engine_idling       2641
children_playing    2639
air_conditioner     2628
drilling            2621
jackhammer          2621
dog_bark            2620
siren               2439
car_horn            1141
Name: count, dtype: int64


In [ ]:
#%%

# Group by noise class to check the distribution of unique noise files per class
unique_noise_files_per_class = metadata_info.groupby('noise_class')['noise_file'].nunique()
print("Unique noise files per class:")
print(unique_noise_files_per_class)

# Check the value counts of noise classes
noise_class_counts = metadata_info['noise_class'].value_counts()
print("\nNoise class counts:")
print(noise_class_counts)

# Check the fold distribution
fold_counts = metadata_info['fold'].value_counts()
print("\nFold distribution:")
print(fold_counts)

# Check the noise file usage distribution
noise_file_usage = metadata_info['noise_file'].value_counts()
print("\nNoise file usage counts:")
print(noise_file_usage)

# Check the unique counts of how many times noise files are used
print("\nUnique counts of noise file usage:")
print(noise_file_usage.unique())

# Descriptive statistics for noise file usage
print("\nDescriptive statistics of noise file usage:")
print(noise_file_usage.describe())

Unique noise files per class:
noise_class
air_conditioner     1000
car_horn             429
children_playing    1000
dog_bark            1000
drilling            1000
engine_idling       1000
jackhammer          1000
siren                929
Name: noise_file, dtype: int64

Noise class counts:
noise_class
engine_idling       2641
children_playing    2639
air_conditioner     2628
drilling            2621
jackhammer          2621
dog_bark            2620
siren               2439
car_horn            1141
Name: count, dtype: int64

Fold distribution:
fold
4     2222
5     2110
3     2079
2     1984
1     1929
10    1856
9     1831
7     1794
6     1778
8     1767
Name: count, dtype: int64

Noise file usage counts:
noise_file
204408-2-0-21.wav    3
62567-5-0-7.wav      3
148632-8-0-13.wav    3
165039-7-8-2.wav     3
165039-7-4-0.wav     3
                    ..
187075-5-0-7.wav     2
88569-2-0-21.wav     2
4201-3-2-0.wav       2
107357-8-0-4.wav     2
162134-7-11-0.wav    2
Name: count, Leng